In [25]:
import pandas as pd
import textwrap


In [6]:
table = pd.read_csv("/home/yhkim/code/JobPT/backend/get_similarity/data/korea_jd_sample.csv")

In [8]:
##기존에는 평문이라 retrieval에 큰 문제는 없었는데 이런 데이터는 일정구분자로 구분해서 평문으로 만들어주는게 좋을듯 하다
##물론 최종적으로는 본문과 같은 형태를 만드는게 추천에 베스트로 생각됨
table.head()

,url,job_category,job_name,title,company_name,company_id,tag_name,tag_id,description,main_work,qualification,preferences,welfare,tech_list
0,https://www.wanted.co.kr/wd/229817,10110,소프트웨어 엔지니어,프론트엔드 엔지니어,엘리스∙서울 강남구∙신입-경력 5년,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 ■ 엘리스LXP • 엘리스 플랫폼 유지 보수 및 스프린트를 통한 기능 구현...,"자격요건 • 5년 미만 개발 경력 • React, TypeScript 기반 웹 프론...","우대사항 • 기본적인 테스트 작성 경험 (Cypress, Jest, React Te...",혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN
1,https://www.wanted.co.kr/wd/229818,10110,소프트웨어 엔지니어,시니어 프론트엔드 엔지니어,엘리스∙서울 강남구∙경력 5-15년,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 개발 스프린트 리더로서 프로젝트 계획 및 관리 • 스쿼드 단위의 팀 운...,"자격요건 • 5년 이상의 프론트엔드 개발 경력 • React, TypeScript ...",우대사항 • 팀리더 또는 파트리더 경험 • 해외개발자와의 협업 경험 • 컴퓨터공학과...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN
2,https://www.wanted.co.kr/wd/229838,10110,소프트웨어 엔지니어,프론트엔드 코어 엔지니어,엘리스∙서울 강남구∙경력 3년 이상,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 엘리스 AI 플랫폼 핵심 개발 업무 ∘ 다양한 플랫폼을 위한 SDK 설...,자격요건 /*해당 포지션은 엘리스 AI 플랫폼의 핵심 기능을 개발하는 코어 엔지니어...,우대사항 • 팀리더 또는 파트리더 경험 • 해외개발자와의 협업 경험 • 온라인 교육...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN
3,https://www.wanted.co.kr/wd/230306,10110,소프트웨어 엔지니어,시니어 백엔드 엔지니어 (Python),엘리스∙서울 강남구∙경력 5년 이상,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 엘리스 AI 플랫폼 핵심 개발 업무 - 실시간으로 협업 가능한 WebS...,자격요건 • Python 백엔드 경력 5년 이상이신 분 • 컴퓨터공학과 또는 관련 ...,우대사항 • 마이크로서비스 아키텍처 기반의 시스템 개발 경험 • Argo CD 등을...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN
4,https://www.wanted.co.kr/wd/223783,10110,소프트웨어 엔지니어,[내집스캔] 부동산 안전도 표준 개발자,한국부동산데이터연구소∙서울 마포구∙경력 3-15년,/company/42012,NaN,NaN,-내집스캔을 소개합니다- 안녕하세요. 저희는 전세사기 예방 서비스 ‘내집스캔’을 만...,주요업무 초기 멤버로서 풀스택 엔지니어 역할을 수행합니다. 특정 레이어에 국한되지 ...,"자격요건 -프론트/백엔드 한쪽에 치우치지 않고, 서비스 전반을 다룰 수 있는 분 -...",우대사항 이런 분을 환영합니다 -새로운 문제를 마주했을 때 피하지 않고 실험하며 답...,NaN,NaN


In [21]:
##retrieval에 중요한 컬럼은 [description, main_work, qualification, preference]으로 생각된다
##자세히하면 좋을것 같기는한데 이거는 데이터 부분 확답을 받고 진행하자
table.columns

Index(['url', 'job_category', 'job_name', 'title', 'company_name',
       'company_id', 'tag_name', 'tag_id', 'description', 'main_work',
       'qualification', 'preferences', 'welfare', 'tech_list'],
      dtype='object')

In [10]:
temp = table.iloc[0]

In [19]:
## 줄글로 된부분은 구분자를 넣어 만들거고 나머지 키워드 부분은 하나로 묶어 청킹 마지막에 붙일 예정
temp

url                             https://www.wanted.co.kr/wd/229817
job_category                                                 10110
job_name                                                소프트웨어 엔지니어
title                                                   프론트엔드 엔지니어
company_name                                   엘리스∙서울 강남구∙신입-경력 5년
company_id                                           /company/1014
tag_name                                                       NaN
tag_id                                                         NaN
description      "Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환...
main_work        주요업무 ■ 엘리스LXP • 엘리스 플랫폼 유지 보수 및 스프린트를 통한 기능 구현...
qualification    자격요건 • 5년 미만 개발 경력 • React, TypeScript 기반 웹 프론...
preferences      우대사항 • 기본적인 테스트 작성 경험 (Cypress, Jest, React Te...
welfare          혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...
tech_list                                                      NaN
Name: 0, dtype: object

In [35]:
sentence = "<chunk_sep>".join([temp["description"], temp["main_work"], temp["qualification"], \
 f"url: {temp['url']}\n\n job_name: {temp['job_name']}\n\n company_name: {temp['company_name']}\n\n welfare: {temp['welfare']}"])

In [36]:
print(sentence)

"Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환을 이끌고, 모두가 AI 시대의 주역이 되도록 돕는 글로벌 AI 기업입니다. /*AI 인프라 혁신을 통한 AI 에코시스템 구축*/ • 최신 GPU(B200·H200) 및 국산 NPU를 탑재한 AI PMDC 통해 고성능 AI 인프라 구축 및 서비스 제공 • 국내 최고 수준의 전력 효율로 자체 구축한 AI 전용 데이터센터와 'AI 인프라-플랫폼-모델-콘텐츠' 통합 AI 생태계를 조성 • 정부 및 공공기관에 안정적인 클라우드 환경 제공이 가능한 보안인증(CSAP) 획득 /*한국어 특화 AI 모델로 교육 혁신을 선도*/ • AI SaaS 솔루션 '엘리스LXP'로 6,200여 기업·기관과 260만 명 학습자 지원 • 자체 개발한 한국어 특화 생성형 AI 모델 라이브러리로 목적별 맞춤 모델 제공 • 전국 초중고 AI 디지털교과서(AIDT)의 인프라·콘텐츠·플랫폼을 통합 공급 /*Product Div.*/ • 엘리스 플랫폼의 UI/UX를 설계하고, Web과 App을 개발 및 고도화합니다. • 우리의 주요 목표는 학생들에게 고품질의 온라인 교육 환경을 제공하기 위해 끊임없이 연구하고 개발하는 것입니다. 라이브 코딩, 라이브 스트리밍, 컨테이너 기반 실습 환경, AI 학습 도우미 등의 최신 기술을 적용합니다. • 단순 기능 개발이 아니라 ‘왜’라는 질문을 주고 받는 최고의 엔지니어/사이언티스트들로 이루어진 팀입니다. • 새로운 프로그래밍 과목(예. 확률론적 언어)을 배우고 지속적인 전문성 향상을 추구합니다. • 백엔드 개발자, 프론트엔드 개발자, 모바일 개발자, 프로덕트 매니저 및 프로덕트 디자이너와 같은 다양한 직무로 구성되어 있습니다. /*프론트엔드 엔지니어*/ • 혁신적인 AI 기반 교육, 평가 및 클라우드 플랫폼의 UI/UX를 개발하고 구현합니다. • 최신 기술 도입을 두려워하지 않는 팀 문화 가운데, 다양한 기술 스택들을 활용하여 글로벌 단위의 프로젝트에 참여할 기회를 가집니다. • A

In [37]:
wrapped_text = textwrap.fill(sentence, width=80)
print(wrapped_text)

"Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환을 이끌고, 모두가 AI 시대의 주역이 되도록 돕는 글로벌
AI 기업입니다. /*AI 인프라 혁신을 통한 AI 에코시스템 구축*/ • 최신 GPU(B200·H200) 및 국산 NPU를 탑재한 AI
PMDC 통해 고성능 AI 인프라 구축 및 서비스 제공 • 국내 최고 수준의 전력 효율로 자체 구축한 AI 전용 데이터센터와 'AI 인프라-
플랫폼-모델-콘텐츠' 통합 AI 생태계를 조성 • 정부 및 공공기관에 안정적인 클라우드 환경 제공이 가능한 보안인증(CSAP) 획득 /*한국어
특화 AI 모델로 교육 혁신을 선도*/ • AI SaaS 솔루션 '엘리스LXP'로 6,200여 기업·기관과 260만 명 학습자 지원 • 자체
개발한 한국어 특화 생성형 AI 모델 라이브러리로 목적별 맞춤 모델 제공 • 전국 초중고 AI 디지털교과서(AIDT)의 인프라·콘텐츠·플랫폼을
통합 공급 /*Product Div.*/ • 엘리스 플랫폼의 UI/UX를 설계하고, Web과 App을 개발 및 고도화합니다. • 우리의 주요
목표는 학생들에게 고품질의 온라인 교육 환경을 제공하기 위해 끊임없이 연구하고 개발하는 것입니다. 라이브 코딩, 라이브 스트리밍, 컨테이너 기반
실습 환경, AI 학습 도우미 등의 최신 기술을 적용합니다. • 단순 기능 개발이 아니라 ‘왜’라는 질문을 주고 받는 최고의
엔지니어/사이언티스트들로 이루어진 팀입니다. • 새로운 프로그래밍 과목(예. 확률론적 언어)을 배우고 지속적인 전문성 향상을 추구합니다. •
백엔드 개발자, 프론트엔드 개발자, 모바일 개발자, 프로덕트 매니저 및 프로덕트 디자이너와 같은 다양한 직무로 구성되어 있습니다. /*프론트엔드
엔지니어*/ • 혁신적인 AI 기반 교육, 평가 및 클라우드 플랫폼의 UI/UX를 개발하고 구현합니다. • 최신 기술 도입을 두려워하지 않는 팀
문화 가운데, 다양한 기술 스택들을 활용하여 글로벌 단위의 프로젝트에 참여할 기회를 가집니다. • 

In [38]:
table["sentence"] = table.apply(lambda x: "<chunk_sep>".join([x["description"], x["main_work"], x["qualification"], \
    f"url: {x['url']}\n\n job_name: {x['job_name']}\n\n company_name: {x['company_name']}\n\n welfare: {x['welfare']}"]), axis=1)

In [43]:
print(textwrap.fill(table["sentence"][0], width=80))

"Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환을 이끌고, 모두가 AI 시대의 주역이 되도록 돕는 글로벌
AI 기업입니다. /*AI 인프라 혁신을 통한 AI 에코시스템 구축*/ • 최신 GPU(B200·H200) 및 국산 NPU를 탑재한 AI
PMDC 통해 고성능 AI 인프라 구축 및 서비스 제공 • 국내 최고 수준의 전력 효율로 자체 구축한 AI 전용 데이터센터와 'AI 인프라-
플랫폼-모델-콘텐츠' 통합 AI 생태계를 조성 • 정부 및 공공기관에 안정적인 클라우드 환경 제공이 가능한 보안인증(CSAP) 획득 /*한국어
특화 AI 모델로 교육 혁신을 선도*/ • AI SaaS 솔루션 '엘리스LXP'로 6,200여 기업·기관과 260만 명 학습자 지원 • 자체
개발한 한국어 특화 생성형 AI 모델 라이브러리로 목적별 맞춤 모델 제공 • 전국 초중고 AI 디지털교과서(AIDT)의 인프라·콘텐츠·플랫폼을
통합 공급 /*Product Div.*/ • 엘리스 플랫폼의 UI/UX를 설계하고, Web과 App을 개발 및 고도화합니다. • 우리의 주요
목표는 학생들에게 고품질의 온라인 교육 환경을 제공하기 위해 끊임없이 연구하고 개발하는 것입니다. 라이브 코딩, 라이브 스트리밍, 컨테이너 기반
실습 환경, AI 학습 도우미 등의 최신 기술을 적용합니다. • 단순 기능 개발이 아니라 ‘왜’라는 질문을 주고 받는 최고의
엔지니어/사이언티스트들로 이루어진 팀입니다. • 새로운 프로그래밍 과목(예. 확률론적 언어)을 배우고 지속적인 전문성 향상을 추구합니다. •
백엔드 개발자, 프론트엔드 개발자, 모바일 개발자, 프로덕트 매니저 및 프로덕트 디자이너와 같은 다양한 직무로 구성되어 있습니다. /*프론트엔드
엔지니어*/ • 혁신적인 AI 기반 교육, 평가 및 클라우드 플랫폼의 UI/UX를 개발하고 구현합니다. • 최신 기술 도입을 두려워하지 않는 팀
문화 가운데, 다양한 기술 스택들을 활용하여 글로벌 단위의 프로젝트에 참여할 기회를 가집니다. • 

In [45]:
table.to_csv("/home/yhkim/code/JobPT/backend/get_similarity/data/korea_jd_sample_processed.csv", index=False)

In [46]:
new_table = pd.read_csv("/home/yhkim/code/JobPT/backend/get_similarity/data/korea_jd_sample_processed.csv")
new_table.head()

,url,job_category,job_name,title,company_name,company_id,tag_name,tag_id,description,main_work,qualification,preferences,welfare,tech_list,sentence
0,https://www.wanted.co.kr/wd/229817,10110,소프트웨어 엔지니어,프론트엔드 엔지니어,엘리스∙서울 강남구∙신입-경력 5년,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 ■ 엘리스LXP • 엘리스 플랫폼 유지 보수 및 스프린트를 통한 기능 구현...,"자격요건 • 5년 미만 개발 경력 • React, TypeScript 기반 웹 프론...","우대사항 • 기본적인 테스트 작성 경험 (Cypress, Jest, React Te...",혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환..."
1,https://www.wanted.co.kr/wd/229818,10110,소프트웨어 엔지니어,시니어 프론트엔드 엔지니어,엘리스∙서울 강남구∙경력 5-15년,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 개발 스프린트 리더로서 프로젝트 계획 및 관리 • 스쿼드 단위의 팀 운...,"자격요건 • 5년 이상의 프론트엔드 개발 경력 • React, TypeScript ...",우대사항 • 팀리더 또는 파트리더 경험 • 해외개발자와의 협업 경험 • 컴퓨터공학과...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환..."
2,https://www.wanted.co.kr/wd/229838,10110,소프트웨어 엔지니어,프론트엔드 코어 엔지니어,엘리스∙서울 강남구∙경력 3년 이상,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 엘리스 AI 플랫폼 핵심 개발 업무 ∘ 다양한 플랫폼을 위한 SDK 설...,자격요건 /*해당 포지션은 엘리스 AI 플랫폼의 핵심 기능을 개발하는 코어 엔지니어...,우대사항 • 팀리더 또는 파트리더 경험 • 해외개발자와의 협업 경험 • 온라인 교육...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환..."
3,https://www.wanted.co.kr/wd/230306,10110,소프트웨어 엔지니어,시니어 백엔드 엔지니어 (Python),엘리스∙서울 강남구∙경력 5년 이상,/company/1014,NaN,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환...",주요업무 • 엘리스 AI 플랫폼 핵심 개발 업무 - 실시간으로 협업 가능한 WebS...,자격요건 • Python 백엔드 경력 5년 이상이신 분 • 컴퓨터공학과 또는 관련 ...,우대사항 • 마이크로서비스 아키텍처 기반의 시스템 개발 경험 • Argo CD 등을...,혜택 및 복지 1. 엘리서가 일잘러로 성장할 수 있도록 적극 지원합니다. • 웰컴키...,NaN,"""Empowering AI, Elice."" 엘리스는 공공·기업 전 영역의 AI 전환..."
4,https://www.wanted.co.kr/wd/223783,10110,소프트웨어 엔지니어,[내집스캔] 부동산 안전도 표준 개발자,한국부동산데이터연구소∙서울 마포구∙경력 3-15년,/company/42012,NaN,NaN,-내집스캔을 소개합니다- 안녕하세요. 저희는 전세사기 예방 서비스 ‘내집스캔’을 만...,주요업무 초기 멤버로서 풀스택 엔지니어 역할을 수행합니다. 특정 레이어에 국한되지 ...,"자격요건 -프론트/백엔드 한쪽에 치우치지 않고, 서비스 전반을 다룰 수 있는 분 -...",우대사항 이런 분을 환영합니다 -새로운 문제를 마주했을 때 피하지 않고 실험하며 답...,NaN,NaN,-내집스캔을 소개합니다- 안녕하세요. 저희는 전세사기 예방 서비스 ‘내집스캔’을 만...
